In [1]:
!pip install librosa

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
os.getcwd()
print(os.listdir())

['.config', 'Dataset', 'Dataset.zip', 'sample_data']


In [4]:
import os
import zipfile

path = os.getcwd()
file_name = "Dataset.zip"
new_path = os.path.join(path, file_name)
with zipfile.ZipFile(new_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.splitext(new_path)[0])
os.chdir(os.path.splitext(new_path)[0])

In [5]:
import os
import zipfile
import pandas as pd
import numpy as np
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
with zipfile.ZipFile('/content/Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dataset')
train_csv_path = '/content/Dataset/Dataset/Train.csv'
test_csv_path = '/content/Dataset/Dataset/Test.csv'
submission_csv_path = '/content/Dataset/Dataset/Submission.csv'
audio_files_path = '/content/Dataset/Dataset/Audio_Files'


In [6]:

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
submission_df = pd.read_csv(submission_csv_path)
print("Train CSV:")
print(train_df.head())
print("\nTest CSV:")
print(test_df.head())
print("\nSubmission CSV:")
print(submission_df.head())


Train CSV:
         New Classification
0  1725_.wav          Major
1  1659_.wav          Minor
2  2029_.wav          Major
3  2093_.wav          Minor
4  1661_.wav          Major

Test CSV:
         New
0  1837_.wav
1  1670_.wav
2  1463_.wav
3  1424_.wav
4  1366_.wav

Submission CSV:
  Classification
0          Major
1          Major
2          Major
3          Major
4          Major


In [7]:
!pip install resampy


In [8]:

!pip install librosa==0.10.2.post1 resampy==0.4.3


Found existing installation: librosa 0.10.2.post1
Uninstalling librosa-0.10.2.post1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/librosa-0.10.2.post1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/librosa/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/audio._zc_wrapper-1158.py310.1.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/audio._zc_wrapper-1158.py310.2.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/audio._zc_wrapper-1158.py310.nbi
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/guf-audio._zc_wrapper-1158.py310.1.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/guf-audio._zc_wrapper-1158.py310.2.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/guf-audio._zc_wrapper-1158.py310.nbi
    /usr/local/lib/python3.10/dist-packages/librosa/util/__pycache__/guf-utils.__peak_pick

In [9]:
import sys
sys.path.append("<resampy_path>")

In [10]:
!pip install librosa resampy --upgrade

import resampy
import librosa
import os
import numpy as np
import pandas as pd

def extract_features(file_name, audio_dir):
    file_path = os.path.join(audio_dir, file_name)
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)

    return mfccs_scaled

train_features = []
train_labels = []

for index, row in train_df.iterrows():
    file_name = row['New']
    label = row['Classification']
    features = extract_features(file_name, audio_files_path)

    train_features.append(features)
    train_labels.append(label)

X_train = np.array(train_features)
y_train = np.array(train_labels)

print("Training features shape:", X_train.shape)
print("Training labels shape:", y_train.shape)


Training features shape: (678, 40)
Training labels shape: (678,)


In [11]:

X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_split, y_train_split)
y_val_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", accuracy)
print(classification_report(y_val, y_val_pred))


Validation Accuracy: 0.7352941176470589
              precision    recall  f1-score   support

       Major       0.73      0.84      0.78        77
       Minor       0.74      0.59      0.66        59

    accuracy                           0.74       136
   macro avg       0.74      0.72      0.72       136
weighted avg       0.74      0.74      0.73       136



In [12]:
test_features = []

for index, row in test_df.iterrows():
    file_name = row['New']
    features = extract_features(file_name, audio_files_path)
    test_features.append(features)

X_test = np.array(test_features)
test_predictions = model.predict(X_test)
submission_df['Classification'] = test_predictions
submission_df.to_csv('Submission.csv', index=False)
print("Predictions saved to Submission.csv")


Predictions saved to Submission.csv


In [13]:
files.download('Submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
def extract_features(file_name, audio_dir):
    file_path = os.path.join(audio_dir, file_name)
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    chroma_scaled = np.mean(chroma.T, axis=0)
    spec_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
    spec_contrast_scaled = np.mean(spec_contrast.T, axis=0)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sample_rate)
    tonnetz_scaled = np.mean(tonnetz.T, axis=0)
    features = np.concatenate((mfccs_scaled, chroma_scaled, spec_contrast_scaled, tonnetz_scaled))

    return features
train_features = []
train_labels = []

for index, row in train_df.iterrows():
    file_name = row['New']
    label = row['Classification']
    features = extract_features(file_name, audio_files_path)

    train_features.append(features)
    train_labels.append(label)

X_train = np.array(train_features)
y_train = np.array(train_labels)

print("Updated Training features shape:", X_train.shape)


Updated Training features shape: (678, 65)


In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
gb_model = GradientBoostingClassifier()
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_gb_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)
print("Best CV Accuracy:", grid_search.best_score_)


Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Best CV Accuracy: 0.7920353982300886


In [16]:
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
best_gb_model.fit(X_train_split, y_train_split)
y_val_pred = best_gb_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Updated Validation Accuracy:", accuracy)
print(classification_report(y_val, y_val_pred))


Updated Validation Accuracy: 0.7573529411764706
              precision    recall  f1-score   support

       Major       0.79      0.78      0.78        77
       Minor       0.72      0.73      0.72        59

    accuracy                           0.76       136
   macro avg       0.75      0.75      0.75       136
weighted avg       0.76      0.76      0.76       136



In [17]:
test_features = []

for index, row in test_df.iterrows():
    file_name = row['New']
    features = extract_features(file_name, audio_files_path)
    test_features.append(features)

X_test = np.array(test_features)\
test_predictions = best_gb_model.predict(X_test)

submission_df['Classification'] = test_predictions
submission_df.to_csv('Submission.csv', index=False)
print("Updated predictions saved to Submission.csv")


Updated predictions saved to Submission.csv


In [18]:

from google.colab import files
files.download('Submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
!pip install xgboost
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier


In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_split_scaled, X_val_scaled = train_test_split(X_train_scaled, test_size=0.2, random_state=42)


In [24]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)


In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_split_scaled, X_val_scaled, y_train_split_encoded, y_val_encoded = train_test_split(
    X_train_scaled, y_train_encoded, test_size=0.2, random_state=42
)
xgb_model = xgb.XGBClassifier(eval_metric="mlogloss", use_label_encoder=False)
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}

grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train_scaled, y_train_encoded)
best_xgb_model = grid_search_xgb.best_estimator_
best_xgb_model.fit(X_train_split_scaled, y_train_split_encoded)
y_val_pred = best_xgb_model.predict(X_val_scaled)
accuracy = accuracy_score(y_val_encoded, y_val_pred)
print("Updated Validation Accuracy with XGBoost:", accuracy)
print(classification_report(y_val_encoded, y_val_pred))


Updated Validation Accuracy with XGBoost: 0.7867647058823529
              precision    recall  f1-score   support

           0       0.82      0.81      0.81        77
           1       0.75      0.76      0.76        59

    accuracy                           0.79       136
   macro avg       0.78      0.78      0.78       136
weighted avg       0.79      0.79      0.79       136



In [25]:
X_test_scaled = scaler.transform(X_test)
test_predictions_encoded = best_xgb_model.predict(X_test_scaled)
test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

submission_df['Classification'] = test_predictions
submission_df.to_csv('Submission.csv', index=False)
print("Updated predictions saved to Submission.csv")

files.download('Submission.csv')



Updated predictions saved to Submission.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
!pip install xgboost
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import xgboost as xgb
import numpy as np


In [27]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_split_scaled, X_val_scaled, y_train_split_encoded, y_val_encoded = train_test_split(
    X_train_scaled, y_train_encoded, test_size=0.2, random_state=42
)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)


In [28]:
xgb_model = xgb.XGBClassifier(eval_metric="mlogloss", use_label_encoder=False)
param_grid_xgb = {
    'n_estimators': [200, 300],
    'max_depth': [5, 7],
    'learning_rate': [0.1, 0.2]
}

grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train_scaled, y_train_encoded)
best_xgb_model = grid_search_xgb.best_estimator_


In [29]:
rf_model = RandomForestClassifier(random_state=42)
param_grid_rf = {
    'n_estimators': [200, 300],
    'max_depth': [10, 15],
    'min_samples_split': [2, 5]
}

grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train_scaled, y_train_encoded)
best_rf_model = grid_search_rf.best_estimator_


In [30]:
ensemble_model = VotingClassifier(
    estimators=[('xgb', best_xgb_model), ('rf', best_rf_model)],
    voting='soft'
)

ensemble_model.fit(X_train_split_scaled, y_train_split_encoded)
y_val_pred = ensemble_model.predict(X_val_scaled)
accuracy = accuracy_score(y_val_encoded, y_val_pred)
print("Validation Accuracy with Ensemble:", accuracy)
print(classification_report(y_val_encoded, y_val_pred))


Validation Accuracy with Ensemble: 0.8014705882352942
              precision    recall  f1-score   support

           0       0.82      0.83      0.83        77
           1       0.78      0.76      0.77        59

    accuracy                           0.80       136
   macro avg       0.80      0.80      0.80       136
weighted avg       0.80      0.80      0.80       136



In [31]:
X_test_scaled = scaler.transform(X_test)
test_predictions_encoded = ensemble_model.predict(X_test_scaled)
test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

submission_df['Classification'] = test_predictions
submission_df.to_csv('Submission.csv', index=False)
files.download('Submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>